In [76]:
from keras.datasets import imdb
from keras.layers import GRU, LSTM, Activation, Dense
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import pandas as pd
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [65]:
num_words = 30000
max_length = 300

In [66]:
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=num_words)

In [67]:
x_val = x_test[:15000]
y_val = y_test[:15000]
x_test = x_test[15000:]
y_test = y_test[15000:]
print(x_train.shape, " ", y_train.shape)
print(x_val.shape, " ", y_val.shape)
print(x_test.shape, " ", y_test.shape)

(25000,)   (25000,)
(15000,)   (15000,)
(10000,)   (10000,)


In [68]:
# pad sequence with zeros
# padding parameters is set to 'post' => zeros are appended to the end of the sequence
x_train = pad_sequences(x_train, maxlen= max_length, padding='post')
x_val = pad_sequences(x_val, maxlen= max_length, padding='post')
x_test = pad_sequences(x_test, maxlen= max_length, padding='post')
print(x_train.shape, " ", x_test.shape)

x_train = x_train.reshape(x_train.shape + (1,))
x_val = x_val.reshape(x_val.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

print(x_train.shape, " ", x_val.shape, " ", x_test.shape)

(25000, 300)   (10000, 300)
(25000, 300, 1)   (15000, 300, 1)   (10000, 300, 1)


In [ ]:
# train
all_train_words = []
for sentence in x_train:
   for word in sentence:
    all_train_words.append(word.item())

In [69]:
print(len(all_train_words))
unique_train_words = set(all_train_words)
print(len(unique_train_words))

7500000
29966


In [70]:
# val
all_val_words = []
for sentence in x_val:
   for word in sentence:
    all_val_words.append(word.item())

In [71]:
print(len(all_val_words))
unique_val_words = set(all_val_words)
print(len(unique_val_words))

4500000
26767


In [72]:
# test
all_test_words = []
for sentence in x_test:
   for word in sentence:
    all_test_words.append(word.item())

In [73]:
print(len(all_test_words))
unique_test_words = set(all_test_words)
print(len(unique_test_words))

3000000
25598


In [83]:
def gru_model():
  model = Sequential()
  model.add(Embedding(30000, 50, input_length=300))
  model.add(GRU(50, input_shape=(300,1), return_sequences=True))
  model.add(GRU(1, return_sequences=False))
  model.add(Activation("sigmoid"))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
  model.summary()
  return model

In [84]:
gru_model = gru_model()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 50)           1500000   
                                                                 
 gru_9 (GRU)                 (None, 300, 50)           15300     
                                                                 
 gru_10 (GRU)                (None, 1)                 159       
                                                                 
 activation_4 (Activation)   (None, 1)                 0         
                                                                 
Total params: 1515459 (5.78 MB)
Trainable params: 1515459 (5.78 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [85]:
#early_stop = EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights = True)
#gru_model.fit(x_train, y_train, validation_data=(x_val,y_val), batch_size=256, epochs=20, verbose=1, callbacks=[early_stop])
gru_model.fit(x_train, y_train, validation_data=(x_val,y_val), batch_size=256, epochs=20, verbose=1)

Epoch 1/20
98/98 [==============================] - 98s 967ms/step - loss: 0.6904 - accuracy: 0.5179 - val_loss: 0.6807 - val_accuracy: 0.5427
Epoch 2/20
98/98 [==============================] - 103s 1s/step - loss: 0.6675 - accuracy: 0.6294 - val_loss: 0.6399 - val_accuracy: 0.6726
Epoch 3/20
98/98 [==============================] - 102s 1s/step - loss: 0.7103 - accuracy: 0.6030 - val_loss: 0.7742 - val_accuracy: 0.5390
Epoch 4/20
98/98 [==============================] - 99s 1s/step - loss: 0.7798 - accuracy: 0.5334 - val_loss: 0.7813 - val_accuracy: 0.5319
Epoch 5/20
98/98 [==============================] - 98s 1s/step - loss: 0.7796 - accuracy: 0.5336 - val_loss: 0.7812 - val_accuracy: 0.5320
Epoch 6/20
98/98 [==============================] - 98s 1s/step - loss: 0.7731 - accuracy: 0.5360 - val_loss: 0.6806 - val_accuracy: 0.5673
Epoch 7/20
98/98 [==============================] - 98s 1s/step - loss: 0.7113 - accuracy: 0.5945 - val_loss: 0.6776 - val_accuracy: 0.6373
Epoch 8/20
98/9

In [86]:
score = gru_model.evaluate(x_test, y_test, verbose=0)
print(score[1]*100)
# needs more training

85.31000018119812


In [169]:
def cnn_model():
  model = Sequential()
  model.add(Embedding(30000, 45, input_length=300))
  model.add(Conv1D(16, 5, activation='relu'))
  model.add(GlobalMaxPooling1D())
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
  model.summary()
  return model

In [170]:
cnn_model = cnn_model()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 300, 45)           1350000   
                                                                 
 conv1d_28 (Conv1D)          (None, 296, 16)           3616      
                                                                 
 global_max_pooling1d_21 (G  (None, 16)                0         
 lobalMaxPooling1D)                                              
                                                                 
 dense_23 (Dense)            (None, 1)                 17        
                                                                 
Total params: 1353633 (5.16 MB)
Trainable params: 1353633 (5.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [171]:
cnn_model.fit(x_train, y_train, validation_data=(x_val,y_val), batch_size=256, epochs=7, verbose=1)

Epoch 1/7
98/98 [==============================] - 16s 151ms/step - loss: 0.6558 - accuracy: 0.6813 - val_loss: 0.5606 - val_accuracy: 0.7785
Epoch 2/7
98/98 [==============================] - 15s 156ms/step - loss: 0.4477 - accuracy: 0.8151 - val_loss: 0.3882 - val_accuracy: 0.8309
Epoch 3/7
98/98 [==============================] - 14s 145ms/step - loss: 0.3130 - accuracy: 0.8772 - val_loss: 0.3302 - val_accuracy: 0.8588
Epoch 4/7
98/98 [==============================] - 14s 147ms/step - loss: 0.2245 - accuracy: 0.9207 - val_loss: 0.3072 - val_accuracy: 0.8693
Epoch 5/7
98/98 [==============================] - 14s 141ms/step - loss: 0.1582 - accuracy: 0.9520 - val_loss: 0.3028 - val_accuracy: 0.8721
Epoch 6/7
98/98 [==============================] - 14s 140ms/step - loss: 0.1087 - accuracy: 0.9716 - val_loss: 0.3068 - val_accuracy: 0.8729
Epoch 7/7
98/98 [==============================] - 14s 141ms/step - loss: 0.0732 - accuracy: 0.9844 - val_loss: 0.3192 - val_accuracy: 0.8703


In [172]:
score = cnn_model.evaluate(x_test, y_test, verbose=0)
print(score[1]*100)
# needs more training

87.6200020313263
